<a href="https://colab.research.google.com/github/tausifshareff/Data-Science/blob/master/Assignment%202/Redo_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk import ngrams
nltk.download('punkt')
nltk.download('stopwords')
import re
from itertools import chain
from sklearn import naive_bayes, metrics
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
# Function to read and process CSV files
def read_file(path, val):
  data = np.loadtxt(path, dtype='str', delimiter='\n')
  array = []
  for i in data:
    i = str(i)
    tmp = str(np.char.strip(np.char.strip(i, '['), ']'))
    quote_rem = re.sub("\'", '', tmp)
    space_rem = re.sub(' ', '', quote_rem)
    final_list = space_rem.split(',')
    array.append(final_list)
  
  return identify_label(array, val)

# Optional function to label the pos and neg corpus
def identify_label(array, val):
  labelled_array = [(x, val) for x in array]
  return labelled_array

# Unused Bigram Generation Function From a Pandas Column
def generate_bigram(corpus,val):
  corpus[0] = corpus[0].apply(lambda x: list(ngrams(x,2)))
  temp = [(x,val) for x in list(chain(*corpus[0]))]
  del corpus
  return pd.DataFrame(temp)

# Unused Unigram Generation Function From a Pandas Column
def generate_unigram(corpus,val):
  corpus[0] = corpus[0].apply(lambda x: list(ngrams(x,1)))
  temp = [(x,val) for x in list(chain(*corpus[0]))]
  del corpus
  return pd.DataFrame(temp)

# Classification Model to Train and Validate Predictions
def build_model(mod, x_train, y_train, x_val, y_val, al):
  modl = mod(alpha = al)
  modl.fit(x_train, y_train)
  y_pred = modl.predict(x_val)
    
  return metrics.accuracy_score(y_pred, y_val)

# Shuffling Rows in a Dataframe
def shuffle_frame(dataframe):
  temp = (dataframe.index.values).copy()
  np.random.shuffle(temp)
  dataframe = dataframe.set_index(temp)
  del temp
  dataframe = dataframe.sort_index()
  dataframe = dataframe.reset_index(drop=True)
  return dataframe

In [0]:
# Reading and merging the CSV files
train_pos = pd.DataFrame(np.array(read_file("drive/Colab Notebooks/MSCI 641/train pos.csv", 1)))
train_neg = pd.DataFrame(np.array(read_file("drive/Colab Notebooks/MSCI 641/train neg.csv", 0)))
test_pos = pd.DataFrame(np.array(read_file("drive/Colab Notebooks/MSCI 641/test pos.csv", 1)))
test_neg = pd.DataFrame(np.array(read_file("drive/Colab Notebooks/MSCI 641/test neg.csv", 0)))
val_pos = pd.DataFrame(np.array(read_file("drive/Colab Notebooks/MSCI 641/val pos.csv", 1)))
val_neg = pd.DataFrame(np.array(read_file("drive/Colab Notebooks/MSCI 641/val neg.csv", 0)))

In [0]:
# Merging positive and negative datasets
train = pd.concat([train_pos, train_neg], axis=0)
test = pd.concat([test_pos, test_neg], axis=0)
val = pd.concat([val_pos, val_neg], axis=0)

# For Unigrams

In [5]:
countvec = CountVectorizer(analyzer="word", ngram_range=(1,1))
countvec.fit([" ".join(x) for x in train[0]])
X_train = countvec.transform([" ".join(x) for x in train[0]])
Y_train = list(train[1].values)

X_val = countvec.transform([" ".join(x) for x in val[0]])
Y_val = list(val[1].values)

X_test = countvec.transform([" ".join(x) for x in test[0]])
Y_test = list(test[1].values)

print("Predicting Unigram Sentiments:")
print("\n")
print("Tuning of alpha in range 0 to 9 in steps 0.2")
tune = [build_model(naive_bayes.MultinomialNB, X_train, Y_train, X_val, Y_val, x) for x in np.arange(0,9,0.2)]
best_param = round(np.arange(0,9,0.2)[tune.index(max(tune))], 2)
print("Best alpha parameter value obtained by tuning on validation data: alpha =", best_param)
print("Test accuracy with tuned alpha value:", round(100*build_model(naive_bayes.MultinomialNB, X_train, Y_train, X_test, Y_test, best_param),2))
print("\n")

Predicting Unigram Sentiments:


Tuning of alpha in range 0 to 9 in steps 0.2


/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:485: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


Best alpha parameter value obtained by tuning on validation data: alpha = 1.2
Test accuracy with tuned alpha value: 72.47




# For Bigrams

In [6]:
countvec = CountVectorizer(analyzer="word", ngram_range=(2,2))
countvec.fit([" ".join(x) for x in train[0]])
X_train = countvec.transform([" ".join(x) for x in train[0]])
Y_train = list(train[1].values)

X_val = countvec.transform([" ".join(x) for x in val[0]])
Y_val = list(val[1].values)

X_test = countvec.transform([" ".join(x) for x in test[0]])
Y_test = list(test[1].values)

print("Predicting Bigram Sentiments:")
print("\n")
print("Tuning of alpha in range 0 to 9 in steps 0.2")
tune = [build_model(naive_bayes.MultinomialNB, X_train, Y_train, X_val, Y_val, x) for x in np.arange(0,9,0.2)]
best_param = round(np.arange(0,9,0.2)[tune.index(max(tune))], 2)
print("Best alpha parameter value obtained by tuning on validation data: alpha =", best_param)
print("Test accuracy with tuned alpha value:", round(100*build_model(naive_bayes.MultinomialNB, X_train, Y_train, X_test, Y_test, best_param),2))
print("\n")

Predicting Bigram Sentiments:


Tuning of alpha in range 0 to 9 in steps 0.2


/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:485: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


Best alpha parameter value obtained by tuning on validation data: alpha = 0.8
Test accuracy with tuned alpha value: 75.11




# For Unigrams+Bigrams



In [7]:
# For Unigrams+Bigrams
countvec = CountVectorizer(analyzer="word", ngram_range=(1,2))
countvec.fit([" ".join(x) for x in train[0]])

X_train = countvec.transform([" ".join(x) for x in train[0]])
Y_train = list(train[1].values)

X_val = countvec.transform([" ".join(x) for x in val[0]])
Y_val = list(val[1].values)

X_test = countvec.transform([" ".join(x) for x in test[0]])
Y_test = list(test[1].values)

print("Predicting Unigram+Bigram Sentiments:")
print("\n")
print("Tuning of alpha in range 0 to 9 in steps 0.2")
tune = [build_model(naive_bayes.MultinomialNB, X_train, Y_train, X_val, Y_val, x) for x in np.arange(0,9,0.2)]
best_param = round(np.arange(0,9,0.2)[tune.index(max(tune))], 2)
print("Best alpha parameter value obtained by tuning on validation data: alpha =", best_param)
print("Test accuracy with tuned alpha value:", round(100*build_model(naive_bayes.MultinomialNB, X_train, Y_train, X_test, Y_test, best_param),2))

Predicting Unigram+Bigram Sentiments:


Tuning of alpha in range 0 to 9 in steps 0.2


/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:485: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


Best alpha parameter value obtained by tuning on validation data: alpha = 0.4
Test accuracy with tuned alpha value: 75.96
